In [1]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count
import pymc as pm
import numpy as np

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
params = {
    "start_population_size": 2300000.,
    "seed_time": 1810.,
    "seed_num": 10.,
    "seed_duration": 3.,
    "progression_multiplier": 1.8,
    "screening_scaleup_shape": 0.1,
    "screening_inflection_time": 1993.,
    "screening_end_asymp": 0.65,
}

bcm = get_bcm(params)

In [4]:
num_cores = cpu_count()
# Set the number of workers for parallel optimization
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class, num_workers=num_cores, ci=0.67)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.006962237803672445,
 'rr_infection_latent': 0.2536223860689803,
 'rr_infection_recovered': 0.34737185555266176,
 'smear_positive_death_rate': 0.4290512333229253,
 'smear_negative_death_rate': 0.028841335859094236,
 'smear_positive_self_recovery': 0.26738283049585143,
 'smear_negative_self_recovery': 0.185673229300567,
 'detection_reduction': 0.7527814440533164}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [8]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [9]:
plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [10]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [14]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=derived_df_0.index, y=derived_df_0['cdr'], mode='lines'))
fig = go.Figure(data=go.Scatter(x=derived_df_0.index, y=derived_df_0['cdr'], mode='lines'))

# Update layout for x-axis range
fig.update_layout(
    xaxis=dict(
        range=[1970, 2025]  # Set the range of x-axis
    )
)

# Show plot
fig.show()

In [12]:
# from autumn.infrastructure.remote import springboard
# rts = springboard.task.RemoteTaskStore()
# rts.cd('projects/tbdynamics/s10041_calibration')
# rts.ls()
# mt = rts.get_managed_task('2024-04-10T1335-d50k-t20k-b20k')
# mt.download_all()